# **Aim of the notebook:**
To check if a sentence is translated with ambiguos relation across Hindi, Bengali, Odia [comparatively low resource], does it make same translation or will it vary? run for 10000 times like an array containing 10,000 same sentences and api hitting for 10,000 times. Only for free services like Google Translate and IndicTrans2. 

## Setup

In [1]:
%%capture
!git clone https://github.com/AI4Bharat/IndicTrans2.git

In [2]:
%%capture
%cd /content/IndicTrans2/huggingface_interface

In [3]:
%%capture
!python3 -m pip install nltk sacremoses pandas regex mock transformers>=4.33.2 mosestokenizer


In [4]:
!python3 -c "import nltk; nltk.download('punkt')"


[nltk_data] Downloading package punkt to /home/sofia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
!python3 -m pip install bitsandbytes scipy accelerate datasets


In [6]:
!python3 -m pip install sentencepiece


In [7]:
print("Installed not all the required packages")

print("Installed all the required packages")

!git clone https://github.com/VarunGumma/IndicTransToolkit
%cd IndicTransToolkit
!python3 -m pip install --editable ./
%cd ..

Installed not all the required packages
Installed all the required packages


fatal: destination path 'IndicTransToolkit' already exists and is not an empty directory.
/home/sofia/multilingual/IndicTransToolkit
Obtaining file:///home/sofia/multilingual/IndicTransToolkit
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/VarunGumma/indic_nlp_library to /tmp/pip-install-3snlp4cx/indic-nlp-library-it2_5d22145e62bf4a6d919c9dc2cd8d4611
  Running command git clone --filter=blob:none --quiet https://github.com/VarunGumma/indic_nlp_library /tmp/pip-install-3snlp4cx/indic-nlp-library-it2_5d22145e62bf4a6d919c9dc2cd8d4611
  Resolved https://github.com/VarunGumma/indic_nlp_library to commit 601521e05ed0ed8f2165ac317a47d186e25b6f0d
  Preparing metadata (setup.py) ... done
  Attempting uninstall: IndicTransToolkit
    Found existing installation: IndicTransToolkit 1.0.2
    Uninstalling IndicTransToolkit-1.0.2:
      Successfully uninstalled IndicTransToolkit-1.0.2
  DEPRECATION: Legacy editable install of IndicTransToolkit==1.0.2 from file:///home/sofia/mu

In [8]:
import pandas as pd
import os

Restart after indic download

# Languages Involved:


In [12]:
# languages = [ 'or', 'hi', 'ta', 'te','bn', 'gu',  'kn', 'ml', 'mr',  'pa'  ]

## Langs and scripts

In [9]:
# create a list with data (just include within bracket): Assamese (asm_Beng)	Kashmiri (Arabic) (kas_Arab)	Punjabi (pan_Guru)
# Bengali (ben_Beng)	Kashmiri (Devanagari) (kas_Deva)	Sanskrit (san_Deva)
# Bodo (brx_Deva)	Maithili (mai_Deva)	Santali (sat_Olck)
# Dogri (doi_Deva)	Malayalam (mal_Mlym)	Sindhi (Arabic) (snd_Arab)
# English (eng_Latn)	Marathi (mar_Deva)	Sindhi (Devanagari) (snd_Deva)
# Konkani (gom_Deva)	Manipuri (Bengali) (mni_Beng)	Tamil (tam_Taml)
# Gujarati (guj_Gujr)	Manipuri (Meitei) (mni_Mtei)	Telugu (tel_Telu)
# Hindi (hin_Deva)	Nepali (npi_Deva)	Urdu (urd_Arab)
# Kannada (kan_Knda)	Odia (ory_Orya)	

lang_script_list = [
    # 'asm_Beng', 'kas_Arab',
                  #    'pan_Guru',
                       'ben_Beng', 
                    #  'kas_Deva', 'san_Deva', 'brx_Deva', 'mai_Deva', 'sat_Olck', 'doi_Deva',
                  #      'mal_Mlym',
                  #    #    #  'snd_Arab',
                  #    #    #    'eng_Latn', 
                  #          'mar_Deva', 
                  #    #    #    'snd_Deva', 'gom_Deva', 'mni_Beng', 
                  #          'tam_Taml', 'guj_Gujr', 
                  #    #    #    'mni_Mtei', 
                  #          'tel_Telu',
                   'hin_Deva', 
                     #    #    'npi_Deva', 'urd_Arab', 
                        #    'kan_Knda', 
                           'ory_Orya'
                           ]



In [10]:
lang_code_map = {
    'eng_Latn': 'en',
    'hin_Deva': 'hi',
    'guj_Gujr': 'gu',
    'kan_Knda': 'kn',
    'mal_Mlym': 'ml',
    'mar_Deva': 'mr',
    'tam_Taml': 'ta',
    'tel_Telu': 'te',
    'pan_Guru': 'pa',
    'ben_Beng': 'bn',
    'ory_Orya': 'or'
}


## IndicTrans2

In [11]:
import torch
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig
# from IndicTransTokenizer import IndicProcessor, IndicTransTokenizer
from IndicTransToolkit import IndicProcessor
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

BATCH_SIZE = 16 # edited from 4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
quantization = None
print(DEVICE)

cuda


In [12]:
def initialize_model_and_tokenizer(ckpt_dir, quantization):
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    tokenizer = AutoTokenizer.from_pretrained(ckpt_dir, trust_remote_code=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    if qconfig == None:
        model = model.to(DEVICE)
        if DEVICE == "cuda":
            model.half()

    model.eval()

    return tokenizer, model


def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip):
    translations = []


    from tqdm import tqdm
    for i in tqdm(range(0, len(input_sentences), BATCH_SIZE)):

    # for i in range(0, len(input_sentences), BATCH_SIZE):
        batch = input_sentences[i : i + BATCH_SIZE]

        # Preprocess the batch and extract entity mappings
        batch = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)

        # Tokenize the batch and generate input encodings
        inputs = tokenizer(
            batch,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(DEVICE)

        # Generate translations using the model
        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1, # TODO temp
            )

        # Decode the generated tokens into text

        with tokenizer.as_target_tokenizer():
            generated_tokens = tokenizer.batch_decode(
                generated_tokens.detach().cpu().tolist(),
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True,
            )

        # Postprocess the translations, including entity replacement
        translations += ip.postprocess_batch(generated_tokens, lang=tgt_lang)

        del inputs
        torch.cuda.empty_cache()

    return translations

In [13]:
en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-1B"  # ai4bharat/indictrans2-en-indic-dist-200M
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir,  quantization)

ip_en_ind = IndicProcessor(inference=True)

In [226]:
# # Initialize model and tokenizer for Indic to English
# indic_en_ckpt_dir = "ai4bharat/indictrans2-indic-en-1B"
# indic_en_tokenizer, indic_en_model = initialize_model_and_tokenizer(indic_en_ckpt_dir, "indic-en", quantization)
# ip_ind_en = IndicProcessor(inference=True)

In [227]:
# CODE for INDIC to INDIC MODEL
# indic_indic_ckpt_dir = "ai4bharat/indictrans2-indic-indic-1B"  # ai4bharat/indictrans2-indic-indic-dist-320M
# indic_indic_tokenizer, indic_indic_model = initialize_model_and_tokenizer(indic_indic_ckpt_dir, quantization)

# ip_indic_indic = IndicProcessor(inference=True)

In [228]:
# src_lang, tgt_lang =  "eng_Latn", "hin_Deva",
# hi_translations = batch_translate(sents, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip_en_ind)


In [20]:
# an array storing sentence "She lives with her aunt since her parents died in an accident." for 10,000 times
sents = ["She lives with her aunt since her parents died in an accident."]*1000

In [21]:
# set to store different translations
translations = {}
for lang_script in lang_script_list:
    src_lang, tgt_lang = "eng_Latn", lang_script
    print(f"Translating {src_lang} to {tgt_lang}")
    transl = batch_translate(sents, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip_en_ind)
    translations[tgt_lang] = set(transl)

Translating eng_Latn to ben_Beng


100%|██████████| 63/63 [00:59<00:00,  1.07it/s]


Translating eng_Latn to hin_Deva


100%|██████████| 63/63 [01:21<00:00,  1.29s/it]


Translating eng_Latn to ory_Orya


100%|██████████| 63/63 [01:08<00:00,  1.09s/it]


In [23]:
translations # even on 10k times, the translations are same

{'ben_Beng': {'তার বাবা-মা একটি দুর্ঘটনায় মারা যাওয়ার পর থেকে সে তার আন্টির সাথে থাকে। '},
 'hin_Deva': {'वह अपनी चाची के साथ रहती है क्योंकि उसके माता-पिता की एक दुर्घटना में मृत्यु हो गई थी। '},
 'ory_Orya': {'ଏକ ଦୁର୍ଘଟଣାରେ ତାଙ୍କ ପିତାମାତାଙ୍କ ମୃତ୍ଯ଼ୁ ପରଠାରୁ ସେ ତାଙ୍କ ମାଉସୀଙ୍କ ସହ ରୁହନ୍ତି | '}}

## Microsoft Phi

In [24]:
!pip install requests uuid

In [25]:
import requests, uuid, json

# Add your key and endpoint
key = "f1d5da8e40cc4634b631f7ec807676e1"
endpoint = "https://api.cognitive.microsofttranslator.com"

# location, also known as region.
# required if you're using a multi-service or regional (not global) resource. It can be found in the Azure portal on the Keys and Endpoint page.
location = "centralindia"

path = '/translate'
constructed_url = endpoint + path

In [26]:


headers = {
    'Ocp-Apim-Subscription-Key': key,
    # location required if you're using a multi-service or regional (not global) resource.
    'Ocp-Apim-Subscription-Region': location,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
}

In [33]:
sents = sents[:500]

In [44]:
# translate sents to all langs
# translations_microsoft = {}
# for lang_script in lang_script_list:
lang_script = 'ory_Orya'
src_lang, tgt_lang = "en", lang_code_map[lang_script]
print(f"Translating {src_lang} to {tgt_lang}")
body = [{'text': sent} for sent in sents]
params = {
        'api-version': '3.0',
        'from': 'en',
        'to': tgt_lang
    }

request = requests.post(constructed_url , params=params, headers=headers, json=body)
response = request.json()
print(response)
translations_microsoft[tgt_lang] = set([sent['translations'][0]['text'] for sent in response])


Translating en to or
[{'translations': [{'text': 'ଏକ ଦୁର୍ଘଟଣାରେ ତାଙ୍କ ପିତାମାତାଙ୍କ ମୃତ୍ୟୁ ହେବା ପରଠାରୁ ସେ ତାଙ୍କ ମାଉସୀଙ୍କ ସହିତ ରୁହନ୍ତି ।', 'to': 'or'}]}, {'translations': [{'text': 'ଏକ ଦୁର୍ଘଟଣାରେ ତାଙ୍କ ପିତାମାତାଙ୍କ ମୃତ୍ୟୁ ହେବା ପରଠାରୁ ସେ ତାଙ୍କ ମାଉସୀଙ୍କ ସହିତ ରୁହନ୍ତି ।', 'to': 'or'}]}, {'translations': [{'text': 'ଏକ ଦୁର୍ଘଟଣାରେ ତାଙ୍କ ପିତାମାତାଙ୍କ ମୃତ୍ୟୁ ହେବା ପରଠାରୁ ସେ ତାଙ୍କ ମାଉସୀଙ୍କ ସହିତ ରୁହନ୍ତି ।', 'to': 'or'}]}, {'translations': [{'text': 'ଏକ ଦୁର୍ଘଟଣାରେ ତାଙ୍କ ପିତାମାତାଙ୍କ ମୃତ୍ୟୁ ହେବା ପରଠାରୁ ସେ ତାଙ୍କ ମାଉସୀଙ୍କ ସହିତ ରୁହନ୍ତି ।', 'to': 'or'}]}, {'translations': [{'text': 'ଏକ ଦୁର୍ଘଟଣାରେ ତାଙ୍କ ପିତାମାତାଙ୍କ ମୃତ୍ୟୁ ହେବା ପରଠାରୁ ସେ ତାଙ୍କ ମାଉସୀଙ୍କ ସହିତ ରୁହନ୍ତି ।', 'to': 'or'}]}, {'translations': [{'text': 'ଏକ ଦୁର୍ଘଟଣାରେ ତାଙ୍କ ପିତାମାତାଙ୍କ ମୃତ୍ୟୁ ହେବା ପରଠାରୁ ସେ ତାଙ୍କ ମାଉସୀଙ୍କ ସହିତ ରୁହନ୍ତି ।', 'to': 'or'}]}, {'translations': [{'text': 'ଏକ ଦୁର୍ଘଟଣାରେ ତାଙ୍କ ପିତାମାତାଙ୍କ ମୃତ୍ୟୁ ହେବା ପରଠାରୁ ସେ ତାଙ୍କ ମାଉସୀଙ୍କ ସହିତ ରୁହନ୍ତି ।', 'to': 'or'}]}, {'translations': [{'text': 'ଏକ ଦୁର୍ଘଟଣାରେ ତାଙ୍କ ପିତାମାତାଙ୍କ ମୃତ୍ୟୁ ହେବା ପରଠ

In [45]:
translations_microsoft

{'bn': {'দুর্ঘটনায় বাবা-মা মারা যাওয়ার পর থেকে সে তার খালার সাথে থাকে।'},
 'hi': {'वह अपनी चाची के साथ रहती है क्योंकि उसके माता-पिता की एक दुर्घटना में मृत्यु हो गई थी।'},
 'or': {'ଏକ ଦୁର୍ଘଟଣାରେ ତାଙ୍କ ପିତାମାତାଙ୍କ ମୃତ୍ୟୁ ହେବା ପରଠାରୁ ସେ ତାଙ୍କ ମାଉସୀଙ୍କ ସହିତ ରୁହନ୍ତି ।'}}

## Google Translate API


In [40]:
# pip install googletrans
!pip install googletrans


In [41]:
from googletrans import Translator


translator = Translator(service_urls=['translate.googleapis.com'])
translator

In [42]:
translator.translate('hello', src='en', dest='hi').text

'नमस्ते'

In [46]:
sents = ["She lives with her aunt since her parents died in an accident."]*10000

In [49]:
# translate sents to all langs
# translations_google = {}
# for lang_script in lang_script_list:
src_lang, tgt_lang = "en", lang_code_map['ory_Orya']
print(f"Translating {src_lang} to {tgt_lang}")
translations_google[tgt_lang] = set([translator.translate(sent, src='en', dest=tgt_lang).text for sent in sents])


Translating en to or


ReadTimeout: The read operation timed out

In [48]:

translations_google

{'bn': {'তার বাবা-মা দুর্ঘটনায় মারা যাওয়ার পর থেকে সে তার খালার সাথে থাকে।'},
 'hi': {'उसके माता-पिता की एक दुर्घटना में मृत्यु हो जाने के बाद से वह अपनी चाची के साथ रहती है।'}}

# Conclusions:

IndicTrans2 translated 1k english sentences to 3 languages [30000 sentences] in 3 minutes
where as Google Translate, for 10k sents is taking approximately 1.5 hours [30 min per lang or 10k sents]
had to run mannualy for odia due to limit error.
Phi could only translate 500 english sentences at a time without giving any type of limit error. SO for 3 languages manually changed lang and did.



Microsoft phi:

 {'bn': {'দুর্ঘটনায় বাবা-মা মারা যাওয়ার পর থেকে সে তার খালার সাথে থাকে।'},
 'hi': {'वह अपनी चाची के साथ रहती है क्योंकि उसके माता-पिता की एक दुर्घटना में मृत्यु हो गई थी।'},
 'or': {'ଏକ ଦୁର୍ଘଟଣାରେ ତାଙ୍କ ପିତାମାତାଙ୍କ ମୃତ୍ୟୁ ହେବା ପରଠାରୁ ସେ ତାଙ୍କ ମାଉସୀଙ୍କ ସହିତ ରୁହନ୍ତି ।'}} 

IndicTrans2:
 {'ben_Beng': {'তার বাবা-মা একটি দুর্ঘটনায় মারা যাওয়ার পর থেকে সে তার আন্টির সাথে থাকে। '},
 'hin_Deva': {'वह अपनी चाची के साथ रहती है क्योंकि उसके माता-पिता की एक दुर्घटना में मृत्यु हो गई थी। '},
 'ory_Orya': {'ଏକ ଦୁର୍ଘଟଣାରେ ତାଙ୍କ ପିତାମାତାଙ୍କ ମୃତ୍ଯ଼ୁ ପରଠାରୁ ସେ ତାଙ୍କ ମାଉସୀଙ୍କ ସହ ରୁହନ୍ତି | '}}

 Google translate:

 {'bn': {'তার বাবা-মা দুর্ঘটনায় মারা যাওয়ার পর থেকে সে তার খালার সাথে থাকে।'},
 'hi': {'उसके माता-पिता की एक दुर्घटना में मृत्यु हो जाने के बाद से वह अपनी चाची के साथ रहती है।'}}
 